# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Baier  ->  J. Baier  |  ['J. Baier']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


P. Mollière  ->  P. Mollière  |  ['P. Mollière']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 79 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.20857
extracting tarball to tmp_2503.20857...

 done.


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2503.20857/ICL_Squad_2.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'style/packages' from 'tmp_2503.20857/style/packages.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'style/settings' from 'tmp_2503.20857/style/settings.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']


...

Found 106 bibliographic references in tmp_2503.21304/protoclusters_q1.bbl.
Retrieving document from  https://arxiv.org/e-print/2503.21368


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Euclid Quick Release Q1}(2025)}]{Q1cite}{Euclid Quick Release Q1}. 2025, \url{https://doi.org/10.57780/esa-2853f3b}
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))


extracting tarball to tmp_2503.21368...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.21600


extracting tarball to tmp_2503.21600... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.20857-b31b1b.svg)](https://arxiv.org/abs/2503.20857) | **Intra-Cluster Light as a Dynamical Clock for Galaxy Clusters: Insights from the MAGNETICUM, IllustrisTNG, Hydrangea and Horizon-AGN Simulations**  |
|| L. C. Kimmig, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *23 pages, 16 figures, submitted to A&A -- comments welcome!*|
|**Abstract**|            As the most massive nodes of the cosmic web, galaxy clusters represent the best probes of structure formation. Over time, they grow by accreting and disrupting satellite galaxies, adding those stars to the brightest cluster galaxy (BCG) and the intra-cluster light (ICL). However, the formation pathways of different galaxy clusters can vary significantly. To inform upcoming large surveys, we aim to identify observables that can distinguish galaxy cluster formation pathways. Using four different hydrodynamical simulations, Magneticum, TNG100 of IllustrisTNG, Horizon-AGN, and Hydrangea, we study how the fraction of stellar mass in the BCG and ICL ($f_{ICL+BCG}$) relates to the galaxy cluster mass assembly history. For all simulations, $f_{ICL+BCG}$ is the best tracer for the time at which the cluster has accumulated 50% of its mass ($z_{f}$), performing better than other typical dynamical tracers, such as the subhalo mass fraction, the halo mass, and the center shift. More relaxed clusters have higher $f_{ICL+BCG}$, in rare cases up to 90%, while dynamically active clusters have lower fractions, down to 20%, which we find to be independent of the exact implemented baryonic physics. We determine the average increase in $f_{ICL+BCG}$ from stripping and mergers to be between 3-4% per Gyr. $f_{ICL+BCG}$ is tightly traced by the stellar mass ratio between the BCG and both the second (M12) and fourth (M14) most massive cluster galaxy. The average galaxy cluster has assembled half of its halo mass by $z_{f}=0.67$ (about 6 Gyr ago), though individual histories vary significantly from $z_{f}=0.06$ to $z_{f}=1.77$ (0.8 to 10 Gyr ago). As all four cosmological simulations consistently find that $f_{ICL+BCG}$ is an excellent tracer of the cluster dynamical state, upcoming surveys can leverage measurements of $f_{ICL+BCG}$ to statistically quantify the assembly of the most massive structures.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.20864-b31b1b.svg)](https://arxiv.org/abs/2503.20864) | **Bar ages derived for the first time in nearby galaxies: Insights on secular evolution from the TIMER sample**  |
|| C. d. Sá-Freitas, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *Accepted for publication in Astronomy & Astrophysics (A&A), 20 pages, 13 Figures, 3 Appendice Figures*|
|**Abstract**|            Once galaxies settle their discs and become self-gravitating, stellar bars can form, driving the subsequent evolution of their host galaxy. Determining the ages of bars can therefore shed light on the epoch of the onset of secular evolution. In this work, we apply the first broadly applicable methodology to derive bar ages to a sample of 20 nearby galaxies. The method is based on the co-eval build-up of nuclear structures and bars and involves using IFS data from the MUSE instrument on VLT to disentangle the SFH of the nuclear disc from the background population. This allows us to derive the formation epoch of the nuclear disc and, thus, of the bar. We estimate the bar formation epoch of nearby galaxies - mostly from the TIMER survey-, creating the largest sample of galaxies with known bar ages to date. We find bar formation epochs between 1 and 13 Gyr ago, illustrating how disc-settling and bar formation are processes that first took place in the early Universe and are still taking place in some galaxies. We infer the bar fraction over cosmological time with our sample, finding remarkable agreement with that obtained from direct studies of galaxies at high redshifts. Additionally, for the first time, we can investigate secular evolution processes taking into account the ages of bars. Our results agree with the scenario in which bars aid the quenching of the host galaxy, with galaxies hosting older bars tending to be more "quenched". We also find that older bars tend to be longer, stronger, and host larger nuclear discs. Furthermore, we find evidence of the nuclear disc stellar mass build-up over time. On the other hand, we find no evidence of downsizing playing a role in bar formation, since we find that bar age is independent of galaxy stellar mass. With the means to estimate bar ages, we can begin to understand better when and how bars shape the observed properties of disc galaxies.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21266-b31b1b.svg)](https://arxiv.org/abs/2503.21266) | **The ESO SupJup Survey VII: Clouds and line asymmetries in CRIRES$^+$ J-band spectra of the Luhman 16 binary**  |
|| S. d. Regt, et al. -- incl., <mark>P. Mollière</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**|            Brown dwarfs at the L-T transition likely experience an inhomogeneous clearing of the clouds in their atmospheres. The resulting surface of thin and thick cloudy patches has been put forward to explain the observed variability, J-band brightening, and re-emergence of FeH absorption. We study the closest brown dwarf binary, Luhman 16A and B, in an effort to constrain their chemical and cloud compositions. As this binary consists of an L7.5 and T0.5 component, we gain insight into the atmospheric properties at the L-T transition. As part of the ESO SupJup Survey, we observed Luhman 16AB at high spectral resolution in the J-band ($1.1-1.4\ \mathrm{\mu m}$) using CRIRES$^+$. To analyse the spectra, we employ an atmospheric retrieval framework, coupling the radiative transfer code petitRADTRANS with the MultiNest sampling algorithm. For both objects, we report detections of H$_2$O, K, Na, FeH, and, for the first time in the J-band, hydrogen-fluoride (HF). The K doublet at $1250\ \mathrm{nm}$ shows asymmetric absorption in the blue line wings, which are reproduced via pressure- and temperature-dependent shifts of the line cores. We find evidence for clouds in both spectra and we place constraints on an FeH-depletion in the Luhman 16A photosphere. The inferred over-abundance of FeH for Luhman 16B opposes its predicted rainout into iron clouds. A two-column model, which emulates the patchy surface expected at the L-T transition, is weakly preferred ($\sim 1.8\sigma$) for component B but disfavoured for A ($\sim 5.5\sigma$). The results suggest a uniform surface on Luhman 16A, which is in good agreement with the reduced variability observed for this L-type component. While the presented evidence is not sufficient to draw conclusions about any inhomogeneity on Luhman 16B, future observations covering a broader wavelength range could help to test the cloud-clearing hypothesis.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21304-b31b1b.svg)](https://arxiv.org/abs/2503.21304) | **Euclid Quick Data Release (Q1). The Euclid view on Planck galaxy protocluster candidates: towards a probe of the highest sites of star formation at cosmic noon**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *18 pages, 11 figures. Submitted to A&A*|
|**Abstract**|            We search for galaxy protoclusters at redshifts $z > 1.5$ in the first data release (Q1) of the $\textit{Euclid}$ survey. We make use of the catalogues delivered by the $\textit{Euclid}$ Science Ground Segment (SGS). After a galaxy selection on the $H_\textrm{E}$ magnitude and on the photometric redshift quality, we undertake the search using the $\texttt{DETECTIFz}$ algorithm, an overdensity finder based on Delaunay tessellation that uses photometric redshift probability distributions through Monte Carlo simulations. In this pilot study, we conduct a search in the 11 $\textit{Euclid}$ tiles that contain previously known $\textit{Planck}$ high star-forming galaxy protocluster candidates and focus on the two detections that coincide with these regions. These counterparts lie at photometric redshifts $z_\textrm{ph}=1.63^{+0.19}_{-0.23}$ and $z_\textrm{ph}=1.56^{+0.18}_{-0.21}$ and have both been confirmed by two other independent protocluster detection algorithms. We study their colours, their derived stellar masses and star-formation rates, and we estimate their halo mass lower limits. We investigate whether we are intercepting these galaxy overdensities in their `dying' phase, such that the high star-formation rates would be due to their last unsustainable starburst before transitioning to groups or clusters of galaxies. Indeed, some galaxy members are found to lie above the main sequence of galaxies (star-formation rate versus stellar mass). These overdense regions occupy a specific position in the dark matter halo mass / redshift plane where forming galaxy clusters are expected to have experienced a transition between cold flows to shock heating in the halo. Finally, we empirically update the potential for galaxy protocluster discoveries at redshift up to $z \simeq3$ (wide survey) and $z \simeq5.5$ (deep survey) with $\textit{Euclid}$ for the next data release (DR1).         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.20949-b31b1b.svg)](https://arxiv.org/abs/2503.20949) | **Rapid Construction of Joint Pulsar Timing Array Datasets: The Lite Method**  |
|| B. Larsen, et al. -- incl., <mark>J. Baier</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *Submitted to MNRAS*|
|**Abstract**|            The International Pulsar Timing Array (IPTA)'s second data release (IPTA DR2) combines observations of 65 millisecond pulsars from 7 radio telescopes spanning decades, aiming to detect nanohertz gravitational waves (GWs). IPTA datasets are complex and take years to assemble, often excluding recent data crucial for low-frequency GW searches. To address this, we introduce the ``Lite'' analysis, a framework that informs the full data combination process. Using a Figure of Merit, we first select individual PTA datasets per pulsar, enabling immediate access to new data and providing an early estimate of fully combined dataset results. Applying this method to IPTA DR2, we create an uncombined dataset (DR2 Lite) and an early-combined subset (EDR2) before constructing the final Full DR2 dataset (IPTA DR2). We find that DR2 Lite can detect the common red noise process seen in Full DR2 but overestimates the amplitude as $A = 5.2^{+1.8}_{-1.7} \times 10^{-15}$ at $\gamma = 13/3$, likely due to unmodeled noise. In contrast, the combined datasets improve spectral characterization, with Full DR2 yielding an amplitude of $A = 4.0^{+1.0}_{-0.9} \times 10^{-15}$ at $\gamma = 13/3$. Furthermore, combined datasets yield higher, albeit small, detection statistics for Hellings-Downs correlations. Looking ahead, the Lite method will enable rapid synthesis of the latest PTA data, offering preliminary GW constraints before full dataset combinations are available while also motivating their construction.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21066-b31b1b.svg)](https://arxiv.org/abs/2503.21066) | **eROSITA clusters dynamical state and their impact on the BCG luminosity**  |
|| A. Zenteno, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *10 pages, 6 figures, accepted in A&A*|
|**Abstract**|            The Spectrum Roentgen Gamma/eROSITA first public release contains 12,247 clusters and groups. We use the offset between the Brightest Cluster Galaxy (BCG) and the X--ray peak (D$_{\rm BCG-X}$) to classify the cluster dynamical state of 3,946 galaxy clusters and groups. The X--ray peaks come from the eROSITA survey while the BCG positions come from the DECaLS DR10 optical data, which includes the DECam eROSITA Survey optical data. We aim to investigate the evolution of the merger and relaxed cluster distributions with redshift and mass, and their impact on the BCG. We model the distribution of D$_{\rm BCG-X}$ as the sum of two Rayleigh distributions representing the cluster's relaxed and disturbed populations, and explore their evolution with redshift and mass. To explore the impact of the cluster's dynamical state on the BCG luminosity, we separate the main sample according to the dynamical state. We define clusters as relaxed if D$_{\rm BCG-X}$ < 0.25$r_{500}$, disturbed if D$_{\rm BCG-X}$>0.5$r_{500}$, and as `diverse' otherwise. We find no evolution of the merging fraction in redshift and mass. We observe that the width of the relaxed distribution to increase with redshift, while the width of the two Rayleigh distributions decreases with mass. The analysis reveals that BCGs in relaxed clusters are brighter than BCGs in both the disturbed and diverse cluster population. The most significant differences are found for high mass clusters at higher redshift. The results suggest that BCGs in low-mass clusters are less centrally bound than those in high-mass systems, irrespective of dynamical state. Over time, BCGs in relaxed clusters progressively align with the potential center. This alignment correlates with their luminosity growth relative to BCGs in dynamically disturbed clusters, underscoring the critical role of the clusters dynamical state in regulating BCG evolution. [Abridged]         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21368-b31b1b.svg)](https://arxiv.org/abs/2503.21368) | **The Jiutian simulations for the CSST extra-galactic surveys**  |
|| J. Han, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-03-28*|
|*Comments*| *simulation data products will be gradually released at this https URL*|
|**Abstract**|            We provide an overview of the Jiutian simulations, a hybrid simulation suite for the China Space Survey Telescope (CSST) extragalactic surveys. It consists of four complementary modules: the primary runs with high resolutions under the fiducial concordance cosmology, the emulator runs exploring the parameter uncertainties around the fiducial cosmology, the reconstruction runs intended for recovering the observed Universe site by site, and the extension runs employing extended cosmologies beyond the standard model. For the primary runs, two independent pipelines are adopted to construct subhaloes and merger trees. On top of them, four sets of mock galaxy light-cone catalogs are produced from semi-analytical models and subhalo abundance matching, providing a variety of observational properties including galaxy SED, emission lines, lensing distortions, and mock images. The 129 emulator runs are used to train the CSST emulator, achieving one percent accuracy in predicting a variety of cosmological statistics over $k\leq 10 h{\rm Mpc}^{-1}$ and $z\leq 2$. The reconstruction runs employ a number of subgrid baryonic models to predict the evolution and galaxy population of certain regions in the real Universe with constrained initial conditions, enabling controlled investigation of galaxy formation on top of structure formation. The extension runs cover models with warm dark matter, $f(R)$ gravity, interacting dark energy, and nonzero neutrino masses, revealing differences in the cosmic structure under alternative cosmological models. We introduce the specifications for each run, the data products derived from them, the corresponding pipeline developments, and present some main tests. Using the primary runs, we also show that the subhalo peak mass functions of different levels are approximately universal. These simulations form a comprehensive and open library for the CSST surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.21600-b31b1b.svg)](https://arxiv.org/abs/2503.21600) | **The Impact of the Hubble Tension on the Evidence for Dynamical Dark Energy**  |
|| Y.-H. Pang, <mark>X. Zhang</mark>, Q.-G. Huang |
|*Appeared on*| *2025-03-28*|
|*Comments*| *5 pages, 1 figures, 3 tables*|
|**Abstract**|            Recent findings from the Dark Energy Spectroscopic Instrument (DESI) Data Release 2 (DR2) favor a dynamical dark energy characterized by a phantom crossing feature. This result also implies a lower value of the Hubble constant, thereby intensifying the so-called Hubble tension. To alleviate the Hubble tension, we consider the early dark energy and explore its impact on the evidence for dynamical dark energy including the Hubble constant calibrated by the SH0ES collaboration. We find that incorporating SH0ES prior with CMB, DESI DR2 BAO and Pantheon Plus/Union3/DESY5 data reduces the preference to dynamical dark energy to $1.8\sigma/1.8\sigma/2.9\sigma$ level, respectively. Our results suggest a potential tension between the Hubble constant $H_0$ of the SH0ES measurement and the phantom-to-quintessence transition in dark energy favored by DESI DR2 BAO data.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.20857/./figures_v1/Fig3_smhm.png', 'tmp_2503.20857/./figures_v1/Fig7_ficb.png', 'tmp_2503.20857/./figures_v1/Fig5_track.png']
copying  tmp_2503.20857/./figures_v1/Fig3_smhm.png to _build/html/
copying  tmp_2503.20857/./figures_v1/Fig7_ficb.png to _build/html/
copying  tmp_2503.20857/./figures_v1/Fig5_track.png to _build/html/
exported in  _build/html/2503.20857.md
    + _build/html/tmp_2503.20857/./figures_v1/Fig3_smhm.png
    + _build/html/tmp_2503.20857/./figures_v1/Fig7_ficb.png
    + _build/html/tmp_2503.20857/./figures_v1/Fig5_track.png
found figures ['tmp_2503.20864/./plots/barAge_mstar.png', 'tmp_2503.20864/./plots/mainResults_lSF.png', 'tmp_2503.20864/./plots/mainResults_hSF.png']
copying  tmp_2503.20864/./plots/barAge_mstar.png to _build/html/
copying  tmp_2503.20864/./plots/mainResults_lSF.png to _build/html/
copying  tmp_2503.20864/./plots/mainResults_hSF.png to _build/html/
exported in  _build/html/2503.20864.md
    + _build/html/tmp_2503.20864/./pl

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\creflastconjunction}{, and\nobreakspace}$
$\newcommand$
$\newcommand{\bibfont}{\aa@bibliographyfont}$
$\newcommand\MSUN{M_{\odot}}$</div>



<div id="title">

# Intra-Cluster Light as a Dynamical Clock for Galaxy Clusters: Insights from the MAGNETICUM, IllustrisTNG, Hydrangea and Horizon-AGN Simulations

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.20857-b31b1b.svg)](https://arxiv.org/abs/2503.20857)<mark>Appeared on: 2025-03-28</mark> -  _23 pages, 16 figures, submitted to A&A -- comments welcome!_

</div>
<div id="authors">

L. C. Kimmig, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** As the most massive nodes of the cosmic web, galaxy clusters represent the best probes of structure formation. Over time, they grow by accreting and disrupting satellite galaxies, adding those stars to the brightest cluster galaxy (BCG) and the intra-cluster light (ICL). However, the formation pathways of different galaxy clusters can vary significantly. To inform upcoming large surveys, we aim to identify observables that can distinguish galaxy cluster formation pathways. Using four different hydrodynamical simulations, Magneticum, TNG100 of IllustrisTNG, Horizon-AGN, and Hydrangea, we study how the fraction of stellar mass in the BCG and ICL ( $f_\mathrm{ICL+BCG}$ ) relates to the galaxy cluster mass assembly history. For all simulations, $f_\mathrm{ICL+BCG}$ is the best tracer for the time at which the cluster has accumulated 50 \% of its mass ( $z_\mathrm{form}$ ), performing better than other typical dynamical tracers, such as the subhalo mass fraction, the halo mass, and the position offset of the cluster mass barycenter to the BCG. More relaxed clusters have higher $f_\mathrm{ICL+BCG}$ , in rare cases up to 90 \% of all stellar mass, while dynamically active clusters have lower fractions, down to 20 \% , which we find to be independent of the exact implemented baryonic physics. We determine the average increase in $f_\mathrm{ICL+BCG}$ from stripping and mergers to be between 3-4 \% per Gyr. $f_\mathrm{ICL+BCG}$ itself is tightly traced by the stellar mass ratio between the BCG and both the second (M12) and fourth (M14) most massive cluster galaxy. The average galaxy cluster has assembled half of its halo mass by $z_\mathrm{form}=0.67$ (about $6 $ Gyr ago), though individual histories vary significantly from $z_\mathrm{form}=0.06$ to $z_\mathrm{form}=1.77$ (0.8 to $10 $ Gyr ago). As all four cosmological simulations consistently find that $f_\mathrm{ICL+BCG}$ is an excellent tracer of the cluster dynamical state, upcoming surveys can leverage measurements of $f_\mathrm{ICL+BCG}$ to statistically quantify the assembly of the most massive structures through cosmic time.

</div>

<div id="div_fig1">

<img src="tmp_2503.20857/./figures_v1/Fig3_smhm.png" alt="Fig7" width="100%"/>

**Figure 7. -** _Left:_ Stellar mass of the central BCG within $100$ kpc versus the total halo mass $M_\mathrm{200c}$ for the four simulations, scaled to a common $h$. Magneticum is plotted in blue, IllustrisTNG in purple, Hydrangea in orange and Horizon-AGN in gold. _Center:_ The same stellar mass plotted against the halo mass as given by $M_\mathrm{500c}$. Also plotted are observations by \citet{demaio18} and \citet{kravtsov18}. _Right:_ The total stellar mass (BCG, ICL and satellites) against $M_\mathrm{500c}$ for the simulations, as well as observations by \citet{andreon:2012,lagana:2013,gonzalez13,budzynski14,kravtsov18,sartoris:2020}.
     (*fig:smhm*)

</div>
<div id="div_fig2">

<img src="tmp_2503.20857/./figures_v1/Fig7_ficb.png" alt="Fig3" width="100%"/>

**Figure 3. -** _Upper panel:_$f_\mathrm{ICL+BCG}$ versus formation redshift $z_\mathrm{form}$, for the galaxy clusters from the Magneticum (blue), TNG100 (purple), Horizon-AGN (gold), and Hydrangea simulations (orange). The gray box displays the Pearson correlation coefficient for each simulation between $f_\mathrm{ICL+BCG}$ and $z_\mathrm{form}$. The closer the absolute value is to unity, the tighter the correlation. We further include best fit lines for each simulation determined via the least squares method (colored lines).
    _Middle panel:_ The center shift between the $3$D gas barycenter and the location of the BCG (normalized by $R_\mathrm{200c}$), plotted as a function of the formation redshift, with the colors the same as the upper panel.
    _Lower panel:_ The fraction of the total halo mass mass which is in all subhalos $f_\mathrm{sub}$(so excluding the central and diffuse halo) versus formation redshift, with colors as in the upper two panels.
     (*fig:dyncor*)

</div>
<div id="div_fig3">

<img src="tmp_2503.20857/./figures_v1/Fig5_track.png" alt="Fig8" width="100%"/>

**Figure 8. -** _Upper row:_ Evolution of the halo mass $M_\mathrm{200,c}$ with redshift $z$ for all clusters in the Magneticum Box2 hr simulation, normalized to the final mass $M_\mathrm{200,c}(z=0)$. The black line shows the median, with the dark shaded area marking the $1\sigma$-bounds and the light shaded area marking the $2\sigma$-bounds. The colored lines and their $1\sigma$-bounds show the evolution of a subset of clusters split by their formation redshift (_left_) or their $f_\mathrm{ICL+BCG}$(_right_) as given in the legend. Black dash-dotted vertical lines mark the redshifts at which the average cluster has accumulated 20\%, 50\%, and 90\% of its total mass (horizontal black lines), marked by $z_\mathrm{20}$, $z_\mathrm{50}$, and $z_\mathrm{90}$, respectively. Note that $z_\mathrm{50}$ is what we define as formation redshift of a cluster.
    _Lower row_: Evolution of $f_\mathrm{ICL+BCG}$ with redshift $z$. As in the upper panel, the black line marks the average evolution of all clusters from Magneticum, with the gray shaded areas showing the $1\sigma$- and $2\sigma$-bounds. The colored lines and regions show the median and $1\sigma$-bounds for the same subsets of galaxy clusters.
     (*fig:tracks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.20857"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# Bar ages derived for the first time in nearby galaxies: Insights on secular evolution from the TIMER sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.20864-b31b1b.svg)](https://arxiv.org/abs/2503.20864)<mark>Appeared on: 2025-03-28</mark> -  _Accepted for publication in Astronomy & Astrophysics (A&A), 20 pages, 13 Figures, 3 Appendice Figures_

</div>
<div id="authors">

C. d. Sá-Freitas, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** Once galaxies settle their discs and become self-gravitating, stellar bars can form, driving the subsequent evolution of their host galaxy. Determining the ages of bars can therefore shed light on the epoch of the onset of secular evolution. In this work, we apply the first broadly applicable methodology to derive bar ages to a sample of 20 nearby galaxies. The method is based on the co-eval build-up of nuclear structures and bars, and involves using integral field spectroscopic (IFS) data from the Multi Unit Spectroscopic Explorer (MUSE) instrument on the Very Large Telescope to disentangle the star formation history of the nuclear disc from the background population. This allows us to derive the formation epoch of the nuclear disc and, thus, of the bar. We estimate the bar formation epoch of nearby galaxies -- mostly from the TIMER survey--, creating the largest sample of galaxies with known bar ages to date. We find bar formation epochs varying between $1$ and $13 \mathrm{Gyr}$ ago, illustrating how disc-settling and bar formation are processes that first took place in the early Universe and are still taking place in some galaxies. We infer the bar fraction over cosmological time with our sample, finding remarkable agreement with that obtained from direct studies of galaxies at high redshifts. Additionally, for the first time we are able to investigate secular evolution processes taking into account the ages of bars. Our results agree with the scenario in which bars aid the quenching of the host galaxy, with galaxies hosting older bars tending to be more "quenched". We also find that older bars tend to be longer, stronger and host larger nuclear discs. \textcolor{black}{Furthermore, we find evidence of the nuclear disc stellar mass build-up over time.} On the other hand, we find no evidence of downsizing playing a role on bar formation, since we find that bar age is independent of galaxy stellar mass. With the means to estimate bar ages, we can begin to understand better when and how bars shape the observed properties of disc galaxies.

</div>

<div id="div_fig1">

<img src="tmp_2503.20864/./plots/barAge_mstar.png" alt="Fig3" width="100%"/>

**Figure 3. -** **Stellar masses as a function of bar ages**. We consider the stellar masses from the S$^4$G survey, obtained with the $3.6\mu\mathrm{m}$ band (\citealp{munoz2015spitzer}) and the bar ages derived in this work.  We bootstrapped our sample $1000$ times in order to robustly estimate the _Pearson_ correlation coefficient and the $p-value$, with associated uncertainties. Lastly, we display the mean errors in the bar age \color{black} and stellar mass \color{black}estimates (black-dot). \color{black} As discussed in munoz2015spitzer, the main source of error in the galaxy stellar mass comes from errors in distance estimates, which translate to $\pm0.32$ dex error in mass. \color{black} Contrary to downsizing predictions, we find no correlation between the two properties, with a weak Pearson coefficient of $r=-0.180\pm0.007$ -- illustrated by the dotted line. (*fig_barAge_mstar*)

</div>
<div id="div_fig2">

<img src="tmp_2503.20864/./plots/mainResults_lSF.png" alt="Fig15" width="100%"/>

**Figure 15. -** Individual measurements of bar age of the low-SF sample. Following the de2023new methodology, we define the bar age as the moment in which the SFH of the nuclear disc (dashed-blue line) overcome the SFH of the main disc (dot-dashed-green line). Additionally, based on the tests performed in de2023new and here, we estimate the systematic measurement error of $^{+1.8}_{-1.4} \mathrm{Gyr}$.  (*fig_lSFmain*)

</div>
<div id="div_fig3">

<img src="tmp_2503.20864/./plots/mainResults_hSF.png" alt="Fig16" width="100%"/>

**Figure 16. -** Same as Fig. \ref{fig_lSFmain}, for the high-SF sub-sample. (*fig_hSFmain*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.20864"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.4}$</div>



<div id="title">

# The ESO SupJup Survey VII: Clouds and line asymmetries in CRIRES$^+$ J-band spectra of the Luhman 16 binary

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.21266-b31b1b.svg)](https://arxiv.org/abs/2503.21266)<mark>Appeared on: 2025-03-28</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

S. d. Regt, et al. -- incl., <mark>P. Mollière</mark>

</div>
<div id="abstract">

**Abstract:** Brown dwarfs at the L-T transition likely experience an inhomogeneous clearing of the clouds in their atmospheres. The resulting surface of thin and thick cloudy patches has been put forward to explain the observed variability, J-band brightening, and re-emergence of FeH absorption. We study the closest brown dwarf binary, Luhman 16A and B, in an effort to constrain their chemical and cloud compositions. As this binary consists of an L7.5 and T0.5 component, we gain insight into the atmospheric properties at the L-T transition. As part of the ESO SupJup Survey, we observed Luhman 16AB at high spectral resolution in the J-band ( $1.1$ -- $1.4 \mathrm{\mu m}$ ) using CRIRES $^+$ . To analyse the spectra, we employ an atmospheric retrieval framework, coupling the radiative transfer code \texttt{petitRADTRANS} with the \texttt{MultiNest} sampling algorithm. For both objects, we report detections of $H_2$ O, K, Na, FeH, and, for the first time in the J-band, hydrogen-fluoride (HF). The K doublet at $1250 \mathrm{nm}$ shows asymmetric absorption in the blue line wings, which are reproduced via pressure- and temperature-dependent shifts of the line cores. We find evidence for clouds in both spectra and we place constraints on an FeH-depletion in the Luhman 16A photosphere. The inferred over-abundance of FeH for Luhman 16B opposes its predicted rainout into iron clouds. A two-column model, which emulates the patchy surface expected at the L-T transition, is weakly preferred ( $\sim$ $1.8\sigma$ ) for component B but disfavoured for A ( $\sim$ $5.5\sigma$ ). The results suggest a uniform surface on Luhman 16A, which is in good agreement with the reduced variability observed for this L-type component. While the presented evidence is not sufficient to draw conclusions about any inhomogeneity on Luhman 16B, future observations covering a broader wavelength range could help to test the cloud-clearing hypothesis.

</div>

<div id="div_fig1">

<img src="tmp_2503.21266/./plots/J_band_profiles.png" alt="Fig7" width="100%"/>

**Figure 7. -** Retrieved vertical profiles of Luhman 16A (solid) and Luhman 16B (dashed). _Left panel_: chemical abundances and the $68$, $95$ and $99.7\%$ confidence envelopes of each species, for both brown dwarfs. The dotted lines show the chemical-equilibrium abundances computed with \texttt{FastChem} ([Kitzmann, Stock and Patzer 2024]()) . With the exception of FeH, the modelled abundances are constant with altitude and thus do not show the drop-offs exhibited by the equilibrium profiles.
    _Middle panel_: inferred temperature profiles and the condensation curves (dotted) of four cloud species  ([Visscher, Lodders and Fegley 2006](), [Visscher, Lodders and Fegley 2010]()) . _Right panel_: grey-cloud opacities retrieved as a function of altitude. The orange and blue shading in the panel gaps indicate the Luhman 16AB photospheres as obtained with the integrated emission contribution functions. (*fig:vertical_profiles*)

</div>
<div id="div_fig2">

<img src="tmp_2503.21266/./plots/J_band_K_shift_param.png" alt="Fig1" width="100%"/>

**Figure 1. -** Variation with temperature of the broadening ($\gamma_W/n_p$, solid) and shift ($d/n_p$, dashed) rates of the $\mathrm{4p^2P_{3/2}-5s}$(green) and $\mathrm{4p^2P_{1/2}-5s}$(purple) lines, perturbed by $H_2$ and He. The van der Waals (vdW) broadening, computed with Eq. \ref{eq:width_Kurucz}, is shown in black for comparison. (*fig:core_power_law*)

</div>
<div id="div_fig3">

<img src="tmp_2503.21266/./plots/J_band_spectrum.png" alt="Fig3" width="100%"/>

**Figure 3. -** CRIRES$^+$ J-band spectra of the Luhman 16 binary. _Top panel_: nine spectral orders covered in the J1226 wavelength setting. The two reddest order-detector pairs are omitted, as described in Sect. \ref{sect:obs_and_red}. _Lower panels_: zoom-ins showing the similar absorption features between the brown dwarfs. (*fig:spectrum*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.21266"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\N}[1]{\textcolor{violet}{#1}}$
$\newcommand{\T}[1]{\textcolor{teal}{#1}}$
$\newcommand{\BLF}[1]{\textcolor{green}{#1}}$
$\newcommand{\HD}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\DS}[1]{\textcolor{brown}{DS: #1}}$
$\newcommand{\OC}[1]{\textcolor{orange}{#1}}$
$\newcommand{\Mh}[0]{M_{\rm h}}$
$\newcommand{\pd}{\phantom{0}}$
$\newcommand{\pa}{\phantom{a}}$
$\newcommand{\orcid}[1]$
$\newcommand{\Planck}{\textit{Planck}}$
$\newcommand{\Herschel}{\textit{Herschel}}$
$\newcommand{\Spitzer}{\textit{Spitzer}}$
$\newcommand{\protoa}{G254\_EUC\_2}$
$\newcommand{\protob}{G257\_EUC\_3}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): The $\Euclid$ view on $\Planck$ galaxy protocluster candidates: towards a probe of the highest sites of star formation at cosmic noon

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.21304-b31b1b.svg)](https://arxiv.org/abs/2503.21304)<mark>Appeared on: 2025-03-28</mark> -  _18 pages, 11 figures. Submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We search for galaxy protoclusters at redshifts $z > 1.5$ in the first data release (Q1) of the $\Euclid$ survey. We make use of the catalogues delivered by the $\Euclid$ Science Ground Segment (SGS), especially the positions, photometry, photometric redshifts, and derived physical parameters. After a galaxy selection on the $\HE$ magnitude and on the photometric redshift quality, we undertake the search using the \texttt{DETECTIFz} algorithm, an overdensity finder based on Delaunay tessellation that uses photometric redshift probability distributions through Monte Carlo simulations.In this pilot study, we conduct a search in the 11 $\Euclid$ tiles that contain previously known $\Planck$ high star-forming galaxy protocluster candidates, covering 2.75 deg $^2$ of the whole 63 deg $^2$ of Q1. We then focus on the two $\Euclid$ protocluster candidates that coincide with these $\Planck$ fields.These counterparts lie at photometric redshifts $z_{\rm ph}=1.63^{+0.19}_{-0.23}$ and $z_{\rm ph}=1.56^{+0.18}_{-0.21}$ . One of them also has partial $\Herschel$ coverage. Both detections have been confirmed by the independent protocluster detection algorithms Poisson Probability Method (PPM) and Monte-Carlo Delaunay Tesselation Field Estimator (MC-DTFE-LoG). We study the colours, the derived stellar masses (regarded as lower limits) and star-formation rates of the detected protoclusters. We also estimate their halo mass lower limits using stellar mass-halo mass relations from the literature.We investigate whether we are intercepting these galaxy overdensities in their `dying' (or `swan song') phase, such that the high star-formation rates would be due to their last unsustainable starburst (or star-formation event) before transitioning and maturing to groups or clusters of galaxies. Some galaxy members are found to lie above the main sequence of galaxies (star-formation rate versus stellar mass). These $\Planck$ and $\Euclid$ overdense regions occupy a position in the dark matter halo mass ( $\Mh$ ) / redshift ( $z$ ) plane at 13.0 $<\log_{10} \Mh<$ 13.5, $1.5<z<$ 2; in this locus, halos of forming galaxy clusters are expected to have experienced a transition between cold flows with no shock heating throughout the halo to shock heating in the halo.Finally, we empirically update the potential for galaxy protocluster discoveries at redshift up to $z \simeq3$ (wide survey) and $z \simeq5.5$ (deep survey) with $\Euclid$ for the next data release (DR1).

</div>

<div id="div_fig1">

<img src="tmp_2503.21304/./figs/new-figures/Mhvszall_arthur_log.png" alt="Fig9" width="100%"/>

**Figure 9. -** Protocluster halo mass ($M_{\rm h}$) versus redshift $z$(notice the log-log scale), colour-coded by star-formation rate (SFR, colour bar). \Euclid protocluster lower limits: triangles, using Method B [S22]([Shuntov, McCracken and Gavazzi (2022)](), see Sect. \ref{sect:mstar_mh} for details.
Circles are protoclusters from the literature: [ and Casey (2016)](); [Polletta, Soucail and Dole (2021)](); [Laporte, Zitrin and Dole (2022)](); [Morishita, Roberts-Borsani and Treu (2023)](); [Morishita, Liu and Stiavelli (2024)](); [Sillassen, Jin and Magdis (2024)](); and [Shimakawa, Pérez-Martínez and Dannerbauer (2024)](). The lines illustrate the different predicted gas-cooling regimes: dotted lines come from [ and Dekel (2006)]()(red dashes) and [Daddi, Delvecchio and Dimauro (2022)]()(blue dashes) and separate loci of cold gas in a hot medium (top right) and hot gas (top left); halo masses below the horizontal red dashed line $M_{\rm shock}$, coming from [ and Dekel (2006)](), are predicted to contain only cold flows with no shock heating within haloes at $z<5$.
 (*fig:arthur_mh_vs_z*)

</div>
<div id="div_fig2">

<img src="tmp_2503.21304/./figs/new-figures/z_hist.png" alt="Fig1" width="100%"/>

**Figure 1. -** Photometric distributions of the \Euclid overdensities outside $\Planck$ beams: red, overdensities with S/N $\geq 2$; and blue, S/N $\geq 3$ outside the $\Planck$ beams. Vertical dashed lines represent the values corresponding to the detections inside the $\Planck$ beams, with $2 \leq$ S/N $<3$(red dashes) or S/N $\geq 3$(blue dashes).
 (*fig:z_hist*)

</div>
<div id="div_fig3">

<img src="tmp_2503.21304/./figs/new-figures/sfr_stellar_mass.png" alt="Fig2" width="100%"/>

**Figure 2. -** Star-formation rate as a function of stellar mass of the \Euclid protocluster candidates. Dots represent overdensities with a signal to noise ratio above 2 or 3 outside the $\Planck$ beams, depending on their colour. The two squares are the protocluster candidates discussed in this pilot study. Histograms for stellar mass (top) and SFR (right) are plotted using only detections outside the $\Planck$ beams.
 (*fig:SFR_vs_Mstar_stat_sample*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.21304"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

412  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

19  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
